## Siamese NN

In [2]:
import os
import tensorflow as tf
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.layers import Dense, Input, Lambda
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.regularizers import l2
from tensorflow.keras.models import load_model
from keras.layers import Input, Lambda, Dense, Dropout, Flatten,Activation, Flatten, Reshape
import numpy as np
import tensorflow.keras.backend as K
from SNN.siamese_network import *
from SNN.siamese_network_parse import PrepareDataSNN
from sklearn.model_selection import train_test_split


## Preparar la data

In [28]:
data = PrepareDataSNN()

# Ruta para guardar los datos
data_dir = "./data_snn"
if not os.path.exists(data_dir):
    os.makedirs(data_dir)

data_a, data_b, labels = data.process()
X1_train, X1_val, X2_train, X2_val, y_train, y_val = train_test_split(
    data_a, data_b, labels, test_size=0.2, random_state=42
)

# Verificar las formas de los arrays
print("Formas de los conjuntos de entrenamiento:")
print(f"X1_train: {X1_train.shape}")
print(f"X2_train: {X2_train.shape}")
print(f"y_train: {y_train.shape}")

print("\nFormas de los conjuntos de validación:")
print(f"X1_val: {X1_val.shape}")
print(f"X2_val: {X2_val.shape}")
print(f"y_val: {y_val.shape}")


Formas de los conjuntos de entrenamiento:
X1_train: (756, 65)
X2_train: (756, 65)
y_train: (756,)

Formas de los conjuntos de validación:
X1_val: (190, 65)
X2_val: (190, 65)
y_val: (190,)


## Inicializar la red siamesa, entrenar y guardar el modelo

In [54]:
# Inicializar y entrenar la red
input_shape = (65,)  # Asumiendo que tus vectores de características tienen 65 elementos
siamese_net = SiameseNeuralNetwork(input_shape)
history = siamese_net.train(X1_train, X2_train, y_train, validation_data=(X1_val, X2_val, y_val))

# Guardar el modelo
siamese_net._save_model("model_1")

Epoch 1/20
24/24 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step - accuracy: 0.5421 - loss: 0.6979 - val_accuracy: 0.7368 - val_loss: 0.6381
Epoch 2/20
24/24 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.5525 - loss: 0.6687 - val_accuracy: 0.8053 - val_loss: 0.5599
Epoch 3/20
24/24 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.7007 - loss: 0.6005 - val_accuracy: 0.8211 - val_loss: 0.4828
Epoch 4/20
24/24 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.7787 - loss: 0.5295 - val_accuracy: 0.8368 - val_loss: 0.4448
Epoch 5/20
24/24 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.8006 - loss: 0.4668 - val_accuracy: 0.8474 - val_loss: 0.4389
Epoch 6/20
24/24 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.8324 - loss: 0.4219 - val_accuracy: 0.8737 - val_loss: 0.3903
Epoch 7/20
24/24 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.8461 - loss: 0.4034 - val_accuracy: 0.8368 - val_loss: 0.4250
Epoch 8/20
24/24 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.8522 - loss: 0.3701 - val_accuracy: 0.8368 - val_loss:

## Cargar el modelo guardado

In [65]:

# Asegúrate de definir la capa Lambda si es necesario
def l1_distance(tensors):
    return K.abs(tensors[0] - tensors[1])

# Al cargar el modelo, pasar la función personalizada como custom_objects
model_ = load_model(
    "models/model_9307.h5", 
    custom_objects={"l1_distance": l1_distance}
)

model_.summary()

Model: "functional_91"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_31      │ (None, 65)        │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ input_layer_32      │ (None, 65)        │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ sequential_10       │ (None, 128)       │    198,016 │ input_layer_31[0… │
│ (Sequential)        │                   │            │ input_layer_32[0… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ lambda_10 (Lambda)  │ (None, 128)       │          0 │ sequential_10[0]… │
│                     │                   │            │ sequential_10[1]… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_43 (Dense)    │ (None, 1)         │        129 │ lambda_10[0][0]   │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 198,147 (774.02 KB)

 Trainable params: 198,145 (774.00 KB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 2 (12.00 B)

## Preparar la nueva data de validacion (Proyectos de C#)

In [31]:
data_val = PrepareDataSNN("data/features_vect_val/")
X1_new, X2_new, y_new = data_val.process()

## Evaluar el modelo con los datos nuevos

In [62]:
loss, accuracy = model_.evaluate([X1_new, X2_new], y_new)

NameError: Exception encountered when calling Lambda.call().

[1mname 'K' is not defined[0m

Arguments received by Lambda.call():
  • inputs=['tf.Tensor(shape=(None, 128), dtype=float32)', 'tf.Tensor(shape=(None, 128), dtype=float32)']
  • mask=['None', 'None']
  • training=False

In [63]:
loss, accuracy = siamese_net.evaluate(X1_new, X2_new, y_new)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - accuracy: 0.7500 - loss: 1.9630
Pérdida en los nuevos datos: 1.9630
Precisión en los nuevos datos: 0.7500


## Hacer una predicción

In [17]:
code1_features = np.random.rand(65)  # Ejemplo de vector de características
code2_features = np.random.rand(65)  # Ejemplo de vector de características

similarity = siamese_net.predict_similarity(code1_features, code2_features)

# Interpretar el resultado
if similarity < 0.5:
    print(f"Los códigos son similares con una probabilidad de {1-similarity:.2f}")
else:
    print(f"Los códigos son diferentes con una probabilidad de {similarity:.2f}")

print(f"La similitud entre los dos códigos es: {similarity}")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step
Los códigos son similares con una probabilidad de 0.59
La similitud entre los dos códigos es: 0.40660813450813293
